In [1]:
import pandas as pd
from pathlib import Path
import os
import re
from unicodedata import normalize
import string
import pickle as pkl

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, mean_absolute_error

# Algorithm
from sklearn.linear_model import LogisticRegression

In [2]:
sample_submission = pd.read_csv("../input/nlp-getting-started/sample_submission.csv")
test_df = pd.read_csv("../input/nlp-getting-started/test.csv")
train_df = pd.read_csv("../input/nlp-getting-started/train.csv")

In [3]:
train_df.head(10)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


# Preprocess Tweet Data

In [4]:
def cleaning(tweet_text, df):
    temp = []
    table = str.maketrans("", "", string.punctuation)
    for tweet in tweet_text:
        # Remove links
        tweet = re.sub(r"http\S+", "", tweet)
        # Remove newline
        tweet = tweet.strip('\n')
        # Remove unicode
        tweet = normalize('NFKD', tweet).encode('ascii','ignore')
        # Remove username
        tweet = re.sub('@[^\s]+','',str(tweet))
        # Remove punctuation and change to lower case
        tweet = tweet.translate(table).lower()
        # Remove 'b' at the begining for binary
        tweet = tweet.replace('b', '', 1)
        # Remove whitespace at start of sentence
        tweet = tweet.strip()
#         # Remove numbers
#         tweet = ''.join([i for i in tweet if not i.isdigit()])
        temp.append(tweet)
    try:
        # Concatenate training with target
        processed_tweets = pd.concat([pd.DataFrame(temp), df['target']], axis=1)
        processed_tweets = pd.DataFrame(processed_tweets)
    except KeyError:
        processed_tweets = pd.DataFrame(temp)
#     print(processed_tweets)
    return processed_tweets   

In [5]:
# Preprocess training and testing tweets
processed_tr_tweets = cleaning(train_df['text'], train_df)
processed_tst_tweets = cleaning(test_df['text'], test_df)

processed_tr_tweets.head(10)

,0,target
0,our deeds are the reason of this earthquake ma...,1
1,forest fire near la ronge sask canada,1
2,all residents asked to shelter in place are be...,1
3,13000 people receive wildfires evacuation orde...,1
4,just got sent this photo from ruby alaska as s...,1
5,rockyfire update california hwy 20 closed in ...,1
6,flood disaster heavy rain causes flash floodin...,1
7,im on top of the hill and i can see a fire in ...,1
8,theres an emergency evacuation happening now i...,1
9,im afraid that the tornado is coming to our area,1


In [6]:
# Checking target occurences

processed_tr_tweets['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

# Tokenization

In [7]:
def vectorize_tweets(count_vect, data):
    vect_tweets = count_vect.fit_transform(data)
    print(type(vect_tweets))
    vect_tweets = vect_tweets.toarray()
    return vect_tweets, count_vect

In [8]:
# Convert a collection of text documents to a matrix of token counts
count_vect = CountVectorizer(analyzer='word', lowercase=False, stop_words='english')
# Combine both train and test
# Prevent unequal length of variables after tokenization
combined_tr_tst = pd.concat([processed_tr_tweets[0], processed_tst_tweets[0]], axis=0)
combined_vect, count_vect = vectorize_tweets(count_vect, combined_tr_tst)
print('length of Vocabulary: {}'.format(len(count_vect.vocabulary_)))

# Check length
len_tr = len(processed_tr_tweets[0])
print('Training length: %d' %len_tr)
len_tst = len(processed_tst_tweets[0])
print('Testing length: %d' %len_tst)
print('Length of train + test: %d' %len(combined_vect))

# Split back to train and test
vect_tweets = combined_vect[:len_tr]
vect_tst_tweets = combined_vect[len_tr:]

print(vect_tweets)

<class 'scipy.sparse.csr.csr_matrix'>
length of Vocabulary: 19789
Training length: 7613
Testing length: 3263
Length of train + test: 10876
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


# Training and Evaluation

In [9]:
# Split training and testing
X_train, X_test, y_train, y_test  = train_test_split(
        vect_tweets, 
        processed_tr_tweets['target'],
        train_size=0.80, 
        random_state=True,
        shuffle=True
)

## Use Logistic Regression

In [10]:
# Train
model_rg = LogisticRegression(solver='liblinear')
model_rg = model_rg.fit(X=X_train, y=y_train)

In [11]:
# Evaluate model
y_pred = model_rg.predict(X_test)
print(len(y_pred))
print(accuracy_score(y_test, y_pred))

1523
0.8036769533814839


## Grid Search

In [12]:
model = LogisticRegression()
param_grid = {
    'solver': ['lbfgs', 'newton-cg', 'sag', 'saga', 'liblinear']
}
gs = GridSearchCV(model, param_grid, n_jobs=4, cv=5)
gs.fit(X_train, y_train)
print(_cv.best_params_)
mse = mean_absolute_error(y_train, gs_cv.predict(X_train))
print("Mean abs error Training : %.4f" % mse)
mse = mean_absolute_error(y_test, gs_cv.predict(X_test))
print("Mean abs error Training : %.4f" % mse)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


NameError: name '_cv' is not defined

# Predict new data

In [13]:
# Predict
new_prediction = model_rg.predict(vect_tst_tweets)
new_prediction = pd.DataFrame(new_prediction)
new_prediction = pd.concat([test_df['id'], new_prediction], axis=1)
print(new_prediction)

         id  0
0         0  1
1         2  1
2         3  1
3         9  1
4        11  1
...     ... ..
3258  10861  1
3259  10865  1
3260  10868  1
3261  10874  1
3262  10875  0

[3263 rows x 2 columns]


In [14]:
def save_submission(new_prediction, fname):
    new_prediction = new_prediction.rename({0: 'target'}, axis=1) 
    new_prediction.to_csv(fname, index=False)
    print(new_prediction)

save_submission(new_prediction, 'submission2.csv')

         id  target
0         0       1
1         2       1
2         3       1
3         9       1
4        11       1
...     ...     ...
3258  10861       1
3259  10865       1
3260  10868       1
3261  10874       1
3262  10875       0

[3263 rows x 2 columns]


# Using Deep Learning

In [15]:
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras import optimizers
import math

In [16]:
def define_model(input_len, output_len):
    n_hidden_1 = math.ceil(input_len / 2)
    n_hidden_2 = math.ceil(n_hidden_1 / 2)
    n_hidden_3 = n_hidden_2
    n_hidden_4 = math.ceil(input_len / 2)

    Inp = Input(shape=(input_len, ))
    x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
    x = Dropout(0.3)(x)
    x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
    x = Dropout(0.3)(x)
    x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
    x = Dropout(0.3)(x)
    x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
    output = Dense(output_len, activation='softmax', name = "Output_Layer")(x)
                
    model = Model(Inp, output)
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    model.summary()
    return model

In [17]:
input_len = X_train.shape[1]
model = define_model(input_len, 2)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 19789)]           0         
_________________________________________________________________
Hidden_Layer_1 (Dense)       (None, 9895)              195822050 
_________________________________________________________________
dropout (Dropout)            (None, 9895)              0         
_________________________________________________________________
Hidden_Layer_2 (Dense)       (None, 4948)              48965408  
_________________________________________________________________
dropout_1 (Dropout)          (None, 4948)              0         
_________________________________________________________________
Hidden_Layer_3 (Dense)       (None, 4948)              24487652  
_________________________________________________________________
dropout_2 (Dropout)          (None, 4948)              0     

In [18]:
def train(model, X_train, X_test, y_train, y_test):
    # Hyperparameters
    learning_rate = 0.1
    adam = keras.optimizers.Adam(lr=learning_rate)

    model.fit(
            X_train, y_train,
            batch_size = 100,
            epochs = 1,
            validation_data=(X_test, y_test),
            shuffle=True
        )
    return model

In [19]:
y_train_np = y_train.to_numpy()
y_test_np = y_test.to_numpy()
model = train(model, X_train, X_test, y_train_np, y_test_np)

Train on 6090 samples, validate on 1523 samples
6090/6090 [==============================] - 150s 25ms/sample - loss: 0.5868 - accuracy: 0.7227 - val_loss: 0.4648 - val_accuracy: 0.8083


In [20]:
# Predict
dl_predictions = pd.DataFrame(model.predict(vect_tst_tweets))
dl_rounded = pd.DataFrame([int(x) for x in dl_predictions[1]])
print(dl_rounded)

      0
0     0
1     0
2     0
3     0
4     0
...  ..
3258  0
3259  0
3260  0
3261  0
3262  0

[3263 rows x 1 columns]


In [21]:
format_predictions = pd.concat([test_df['id'], dl_rounded], axis=1)
print(format_predictions)

# save_submission(format_predictions, 'submission3.csv')

         id  0
0         0  0
1         2  0
2         3  0
3         9  0
4        11  0
...     ... ..
3258  10861  0
3259  10865  0
3260  10868  0
3261  10874  0
3262  10875  0

[3263 rows x 2 columns]
